📁 СТРУКТУРНЫЙ АНАЛИЗ КОДА СИСТЕМЫ
ML‑система анализа лояльности клиентов интернет‑магазина
Автор: Глеб Фанасов

1. ИМПОРТ БИБЛИОТЕК

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")


📌 Краткое объяснение
pandas / numpy — основа для работы с данными.

matplotlib / seaborn — визуализация результатов.

RandomForestClassifier — основной ML‑алгоритм.

train_test_split — разделение данных на обучение и тест.

✅ 2. ГЕНЕРАЦИЯ РЕАЛИСТИЧНЫХ ДАННЫХ

In [ ]:
def prepare_klient_data():
    klienty = []

    for i in range(5000):
        vozrast = np.random.randint(18, 70)
        pokupki = np.random.randint(0, 40)
        sredniy_chek = np.random.uniform(200, 30000)
        akcii = np.random.choice([0, 1])
        otsenka = np.random.uniform(1, 5)

        prob = (
            0.30 * (pokupki / 40) +
            0.40 * (sredniy_chek / 30000) +
            0.20 * (otsenka / 5) +
            0.10 * akcii
        )

        prob += np.random.normal(0, 0.08)
        prob = max(0, min(1, prob))
        loyal = np.random.choice([1, 0], p=[prob, 1 - prob])

        klienty.append({
            "Возраст": vozrast,
            "Покупки": pokupki,
            "Средний чек (₽)": sredniy_chek,
            "Акции": akcii,
            "Оценка": otsenka,
            "Лояльный": loyal
        })

    return pd.DataFrame(klienty)


📌 Краткое объяснение
Генерируется 5000 клиентов — достаточно для обучения модели.

Лояльность формируется как вероятность, а не формула → реалистично.

Добавлен шум → модель не будет идеальной (как в реальном бизнесе).

Признаки выбраны так, как это делают маркетологи: покупки, чек, оценка, акции.

Итог — натуральный датасет, похожий на данные интернет‑магазина.

In [ ]:
def create_visualizations(results, n_new):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(f"Анализ {n_new} клиентов", fontsize=16, weight='bold')

    counts = results['Статус'].value_counts()
    colors = ['#4CAF50' if s == 'Лояльный' else '#FF9800' for s in counts.index]
    axes[0].pie(counts.values, labels=counts.index, autopct='%1.1f%%',
                colors=colors, startangle=90)
    axes[0].set_title("Распределение клиентов")

    axes[1].hist(results['Средний чек (₽)'], bins=10, color='#2196F3', alpha=0.7, edgecolor='black')
    axes[1].axvline(results['Средний чек (₽)'].mean(), color='red', linestyle='dashed')
    axes[1].set_title("Распределение среднего чека (₽)")
    axes[1].set_xlabel("Средний чек (₽)")
    axes[1].set_ylabel("Количество клиентов")

    plt.tight_layout()
    plt.show()


📌 Краткое объяснение
Круговая диаграмма показывает долю лояльных и рисковых клиентов.

Гистограмма среднего чека помогает понять платёжеспособность аудитории.

Красная линия — средний чек → ключевой бизнес‑показатель.

Визуализация помогает менеджеру быстро понять ситуацию.

✅ 4. РЕКОМЕНДАЦИИ

In [ ]:
def print_recommendations(results, n_new):
    risk_count = (results['Статус'] == 'В зоне риска').sum()

    print(f"\n📈 ОБЩАЯ СТАТИСТИКА:")
    print(f"Всего проанализировано: {n_new} клиентов")
    print(f"В зоне риска: {risk_count} ({risk_count/n_new:.1%})")

    print(f"\n🎯 РЕКОМЕНДАЦИИ:")
    if risk_count == 0:
        print("✅ Отличные показатели! Все клиенты лояльны.")
        return

    print(f"⚠️ Требуется внимание к {risk_count} клиентам:")
    risk_clients = results[results['Статус'] == 'В зоне риска']

    for idx, client in risk_clients.iterrows():
        print(f"\n   👤 Клиент {client['ID']}:")
        issues = []

        if client['Покупки'] < 3:
            issues.append("мало покупок")
        if client['Средний чек (₽)'] < 2000:
            issues.append("низкий средний чек")
        if client['Оценка'] < 3:
            issues.append("низкая оценка")

        if issues:
            print(f"      • Проблемы: {', '.join(issues)}")
            print("      • Рекомендуется: скидка 5–10%, бонусы, персональное предложение")
        else:
            print("      • Общая низкая активность")


📌 Краткое объяснение
Система автоматически выявляет причины риска.

Рекомендации основаны на реальных практиках e‑commerce: — скидки, бонусы, акции, персональные предложения.

Это превращает модель в полноценный инструмент удержания клиентов.

✅ 5. ОБУЧЕНИЕ МОДЕЛИ

In [ ]:
data = prepare_klient_data()
X = data[['Возраст', 'Покупки', 'Средний чек (₽)', 'Акции', 'Оценка']]
y = data['Лояльный']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)


📌 Краткое объяснение
Random Forest выбран, потому что: ✅ устойчив к шуму ✅ работает с нелинейностями ✅ не требует нормализации ✅ легко интерпретируется

200 деревьев → баланс между скоростью и качеством.

Точность 60,7% — реалистичная, не завышенная.

✅ 6. АНАЛИЗ НОВЫХ КЛИЕНТОВ

In [ ]:
n_new = int(input("\nСколько клиентов вы хотите проанализировать? "))
...
probs = model.predict_proba(new_df[['Возраст', 'Покупки', 'Средний чек (₽)', 'Акции', 'Оценка']])[:, 1]
statuses = ["Лояльный" if p >= 0.5 else "В зоне риска" for p in probs]


📌 Краткое объяснение
Пользователь вводит данные клиента вручную.

Модель выдаёт вероятность лояльности — это лучше, чем просто 0/1.

Порог 0.5 — стандарт для бинарной классификации.

Система работает как интерактивный аналитический инструмент.

✅ 7. ВЫВОД РЕЗУЛЬТАТОВ

In [ ]:
print(results.to_string(index=False))
create_visualizations(results, n_new)
print_recommendations(results, n_new)


📌 Краткое объяснение
Таблица → подробный отчёт.

Графики → визуальная аналитика.

Рекомендации → actionable insights (действия, а не просто цифры).